Example Script Associated with:
# Speed of sound of pure water to 700 MPa and an equation of state to 2300 MPa
## by O. Bollengier, J.M. Brown, and G. Shaw

The following demonstrates the use of a "Local Basis Function" (LBF) representation for the Gibbs energy equation of state of water.   

See [the LocalBasisFunction repository](https://github.com/jmichaelb/LocalBasisFunction/tree/master/Python) for installation requirements.

Units: MPa for pressure, K for temperature, S.I. units for density, Cp, and G (kg, J, m)

Each individual code block below can be executed by clicking the play icon to the left of the block, or by using the Run button at the top. Comments in the code are preceded by a hash.

### 1. Load data into the environment and describe the content

The LBF is stored as a struct in a MATLAB data file, with the range of coverage incorporated into the name.  This struct is loaded into the Python environment as a dictionary, using only those elements needed for evaluation of the spline.  The contents of the spline are then shown. 

Keys in the dictionary match the standard element names used by MATLAB representations of tensor splines. See [MATLAB documentation](https://www.mathworks.com/help/curvefit/multivariate-tensor-product-splines.html) for more information.  

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
from mlbspline import load
from lbftd import statevars, loadGibbs as lg, evalGibbs as eg

G_H2O_2GPa_500K = load.loadSpline('../G_H2O_2GPa_500K.mat')
G_H2O_2GPa_500K

{'form': 'B-',
 'knots': array([array([   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,   15.        ,   20.        ,
          25.        ,   30.        ,   35.        ,   40.        ,
          45.        ,   50.        ,   55.        ,   60.        ,
          65.        ,   70.        ,   75.        ,   80.        ,
          85.        ,   90.        ,   95.09172249,  100.33476036,
         105.79195946,  111.52959614,  117.61756454,  124.03785154,
         130.80859712,  137.94893146,  145.47902897,  153.42016533,
         161.79477754,  170.62652738,  179.94036821,  189.76261551,
         200.12102122,  211.04485216,  222.56497269,  234.71393195,
         247.52605581,  261.03754386,  275.28657167,  290.31339869,
         306.16048195,  322.87259607,  340.49695973,  359.08336909,
         378.6843385 ,  399.35524887,  421.15450412,  444.14369622,
         468.38777922,  493.95525276,  520.91835557,  549.35326967,
         579.3403

### Extract thermodynamic properties from the LBF representation

The thermodynamic variables currently supported for pure substances are listed below.  For each tuple, the second value is descriptive only; the first value is what would be passed to the evaluation function to limit which variables are calculated. 

In [11]:
[(sv.name, sv.calcFn.__name__[4:], ) for sv in statevars.statevars if not sv.reqM]

[('G', 'GibbsEnergy'),
 ('rho', 'Density'),
 ('vel', 'SoundSpeed'),
 ('Cp', 'IsobaricSpecificHeat'),
 ('Cv', 'IsochoricSpecificHeat'),
 ('alpha', 'ThermalExpansivity'),
 ('U', 'InternalEnergy'),
 ('H', 'Enthalpy'),
 ('S', 'Entropy'),
 ('Kt', 'IsothermalBulkModulus'),
 ('Kp', 'IsothermalBulkModulusWrtPressure'),
 ('Ks', 'IsotropicBulkModulus'),
 ('mus', 'SoluteChemicalPotential'),
 ('Vm', 'PartialMolarVolume'),
 ('Cpm', 'PartialMolarHeatCapacity'),
 ('V', 'Volume')]

To calculate thermodynamic variables for a grid of pressures and temperatures, basic syntax is

    output = eg.evalSolutionGibbsGrid(gibbsSp, PTM, *tdvSpec)

where
* ```gibbsSp``` is a Gibbs energy LBF with dimensions pressure (MPa), temperature (K), and (optionally) molality (mol/kg), in the order defined by ```statevars.iP```, ```statevars.iT```, and ```statevars.iM```. If molality is not provided (as is the case here), this function assumes that it is calculating thermodynamic properties for a pure substance.
* ```PTM``` is a Numpy ndarray of ndarrays with the conditions at which ```gibbsSp``` should be evaluated. The number of dimensions must be same as in the spline (```PTM.size == gibbsSp['number'].size```), and each of the inner ndarrays represents one of pressure (P), temperature (T), or molality (M) and must be in the same order and units as in the ```gibbsSp``` parameter. Additionally, each dimension must be sorted from low to high values.
* ```tdvSpec``` is an optional list of the thermodynamic variables (tdvs) to be calculated.  If no values are provided, this function will calculate all supported tdvs (see above).  An unsupported value will result in an error.  See [the LocalBasisFunction source code repository](https://github.com/jmichaelb/LocalBasisFunction/tree/master/Python) for units of each tdv.

To evaluate the LBF for scattered PT points, see documentation for function ```eg.evalSolutionGibbsScatter```.

The output of the function call is a named tuple with the requested thermodynamic variables as named properties matching the statevars requested in the ```*tdvSpec``` parameter . The output also includes a PTM property so that the regime for each tdv is easily passed along with the calculated values.

This call calculates a subset of supported thermodynamic variables (tdvs): 
* density (rho)
* isobaric specific heat (Cp)
* isothermal bulk modulus (Kt)
* thermal expansivity (alpha)

When specifying a subset of supported tdvs, note that the values are case-sensitive.

In [42]:
P = np.linspace(.1, 1500, num=200)
T = np.linspace(240, 500, num=200)
output = eg.evalSolutionGibbsGrid(G_H2O_2GPa_500K, np.array([P, T]), 'rho', 'Cp', 'Kt', 'alpha')

Output values can be addressed using object/property syntax.  Each tdv is a matrix, with dimensions corresponding to pressure and temperature as determined by ```statevars.iP``` and ```statevars.iT```.  

In [43]:
pi = np.random.randint(0,output.PTM[statevars.iP].size)
ti = np.random.randint(0,output.PTM[statevars.iT].size)
"P={:.4f} MPa, T={:.4f} K, density={:.1f} kg/m^3".format(output.PTM[statevars.iP][pi], output.PTM[statevars.iT][ti], output.rho[pi,ti])


'P=761.3558 MPa, T=246.5327 K, density=1229.7 kg/m^3'

For comparison, here is the same information calculated from an LBF based on IAPWS-95.  See [doi:10.1016/j.fluid.2018.02.001](https://doi.org/10.1016/j.fluid.2018.02.001) for details on the differences between this and the actual IAPWS 95 values.

In [46]:
iapws_sp = load.loadSpline('./IAPWS_sp_strct.mat')
iapws_out = eg.evalSolutionGibbsGrid(iapws_sp, np.array([P, T]), 'rho', 'Cp', 'Kt', 'alpha')
"P={:.4f} MPa, T={:.4f} K, density={:.1f} kg/m^3".format(iapws_out.PTM[statevars.iP][pi], iapws_out.PTM[statevars.iT][ti], iapws_out.rho[pi,ti])


'P=761.3558 MPa, T=246.5327 K, density=1225.9 kg/m^3'